In [236]:
import os
import time
import gc
import json
import requests
from datetime import date, timedelta, datetime

import pandas as pd
import numpy as np
import io
from pandas.io.json import json_normalize

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.html.use_mathjax', False) # от italic шрифтов

In [124]:
mydate = '01/10/2019'


In [145]:
l = datetime.strptime(mydate, '%m/%d/%Y')

TypeError: strptime() takes exactly 2 arguments (1 given)

In [137]:
print('{:%d, %b %Y}'.format(l))

10, Jan 2019


In [144]:
datetime.strftime(l, '%d, %b %Y')

'10, Jan 2019'

In [ ]:
from datetime import date, timedelta, datetime

sdate = date(2008, 8, 15)   # start date
edate = date(2008, 9, 15)   # end date

delta = edate - sdate       # as timedelta
start_date = datetime.strptime(start_date, '%m/%d/%Y')
end_date = datetime.strptime(end_date, '%m/%d/%Y')
delta = end_date - start_date
for i in range(delta.days+1):
    day = start_date + timedelta(days=i-1)
    date_day = datetime.strftime(day, '%m/%d/%Y')
    response = requests.get('https://resultsdb-api.rotogrinders.com/api/contests?start='+date_day+'&lean=true')
    data = response.json()
    json_arr.append(data)
    with open('RG_{start_date}_{end_date}.json'.format(start_date, end_date), 'a') as outfile:
        json.dump(json_arr, outfile)

In [ ]:
date = '10/24/2019'

In [232]:
def get_data_rg(start_date, end_date):
    '''FORMAT: MM/DD/YYYY eg. '10/24/2019'
    '''
    json_arr = []
    sdate = datetime.strptime(start_date, '%m/%d/%Y')
    edate = datetime.strptime(end_date, '%m/%d/%Y')
    delta = edate - sdate
    for i in range(delta.days+1):
        day = sdate + timedelta(days=i)
        date_day = datetime.strftime(day, '%m/%d/%Y')
        response = requests.get('https://resultsdb-api.rotogrinders.com/api/contests?start='+date_day+'&lean=true')
        data = response.json()
        #json_arr = json_arr + data
    #return json_arr
        with open('RG_{}_{}.json'.format(sdate, edate), 'a') as outfile:
            json.dump(data, outfile)

'''def map_sport(sportname):
    sportname = sportname.lower()
    if sportname == 'nfl':
        return 1
    if sportname == 'mlb':
        return 2
    if sportname == 'nba':
        return 3
    if sportname == 'nhl':
        return 4
    if sportname == 'golf' or sportname == 'apt':
        return 6
    return ValueError('Sport type is not tracked')
'''

def df_prep(json_data):
    df = pd.DataFrame(json_data, columns = ['name',
                                            'sport',
                                            'gameCount',
                                            'start',                                       
                                            'prizePool',
                                            'entryFee',
                                            'maxEntriesPerUser',
                                            'maxEntries',
                                            'entryCount'
                                           ])
    df['sport'] = df['sport'].map({1:'NFL',
                                   2:'MLB',
                                   3:'NBA',
                                   4:'NHL',
                                   6:'GOLF'})
    df['start'] = df['start'].apply(lambda x:x[:-1])
    df['start'] = pd.to_datetime(df['start'])
    df.insert(3, 'dayofweek', df['start'].dt.day_name())
    df.dropna(inplace=True) # Если нет mE и mEPerUser, на сайте игр нет, хотя в логах played=true. Если нет gameCount - отмененные. 
    df['maxEntries'] = df['maxEntries'].astype('int')
    df['maxEntriesPerUser'] = df['maxEntriesPerUser'].astype('int')
    df['name'] = df['name'].str.lower()
    return df

def rg_df(PATH):
    '''Supported sport types are:
    NFL, NBA, MLB, NHL, Golf
    '''
    with open(PATH, encoding="utf8") as f:
        data = f.readlines()
        data = [json.loads(line) for line in data] #convert string to dict format
    df = df_prep(data[0])
    return df

In [208]:
get_data_rg('10/24/2019', '11/07/2019')

In [234]:
df = rg_df('RG_10-24-19_11-07-19.json')

In [ ]:
#df[df.gameCount.isna() == True]

In [227]:
df[df.isnull().any(axis=1)]

,name,sport,gameCount,dayofweek,start,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
57,$40k thu nhl breakaway ($8k to 1st),NHL,10.00,Thursday,2019-10-24 23:00:00,40000.00,4.44,nan,nan,10725
58,"$5k thu nhl big $2 double up (top 1,250 win)",NHL,10.00,Thursday,2019-10-24 23:00:00,4999.00,2.00,nan,nan,2873
59,$12k thu nhl goal ($2k to 1st),NHL,10.00,Thursday,2019-10-24 23:00:00,12000.00,33.00,nan,nan,427
60,$5k thu nhl wrister (20 entries max),NHL,10.00,Thursday,2019-10-24 23:00:00,5000.00,2.00,nan,nan,2743
61,$10k thu nhl monster,NHL,10.00,Thursday,2019-10-24 23:00:00,10000.00,333.00,nan,nan,33
62,$8k thu nba piggy bank shot,NBA,3.00,Thursday,2019-10-24 23:00:00,7997.00,0.05,nan,nan,177979
63,$16k thu nba dribbler (single entry),NBA,3.00,Thursday,2019-10-24 23:00:00,16000.00,1.00,nan,nan,19047
64,$400k thu nba clutch shot ($100k to 1st),NBA,3.00,Thursday,2019-10-24 23:00:00,399999.00,4.44,nan,nan,107250
65,$30k thu nba fadeaway ($0.25 to enter),NBA,3.00,Thursday,2019-10-24 23:00:00,30002.00,0.25,nan,nan,141187
66,$10k thu nba pick n roll (20 entries max),NBA,3.00,Thursday,2019-10-24 23:00:00,10000.00,2.22,nan,nan,5362


In [222]:
nfl = df[df.sport == 'NFL']
nhl = df[df.sport == 'NHL']
golf = df[df.sport == 'GOLF']
mlb = df[df.sport == 'MLB']
nba = df[df.sport == 'NBA']

In [242]:
labels = np.unique(df.sport.values).tolist()
labels

['GOLF', 'MLB', 'NBA', 'NFL', 'NHL']

In [263]:
gc = df.groupby("sport")['gameCount'].sum()
gc

sport
GOLF     26.00
MLB      63.00
NBA    5495.00
NFL    5751.00
NHL    1071.00
Name: gameCount, dtype: float64

In [264]:
pp = df.groupby("sport")['prizePool'].sum()

In [279]:
labels = np.unique(df.sport.values).tolist()
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen', 'lightblue']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=[gc[0], gc[1], gc[2], gc[3], gc[4]], name='Overall Tournaments Played'),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=[pp[0], pp[1], pp[2], pp[3], pp[4]], name='Overall prizepool'),
              1, 2)

fig.update_traces(hole=.4, hoverinfo='label+name+value', marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(
        title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_text='2019 Sport types comparsion',
    annotations=[dict(text='Tourney count', x=0.14, y=0.5, font_size=20, showarrow=False),
                 dict(text='Prizes sum', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [223]:
nfl.head(5)

,name,sport,gameCount,dayofweek,start,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
95,nfl showdown $80k bootleg [$20k to 1st] (was v...,NFL,1.00,Friday,2019-10-25 00:20:00,80000.00,33.00,84.00,2827.00,2828
96,nfl showdown $95k luxury box [$50k to 1st] (wa...,NFL,1.00,Friday,2019-10-25 00:20:00,95000.00,1500.00,2.00,67.00,68
97,nfl showdown $150k deep threat [$50k to 1st] (...,NFL,1.00,Friday,2019-10-25 00:20:00,150000.00,180.00,27.00,925.00,926
98,nfl $200k thursday night showdown [$50k to 1st...,NFL,1.00,Friday,2019-10-25 00:20:00,200000.00,10.00,150.00,23529.00,23530
99,nfl showdown $60k play-action [20 entry max] (...,NFL,1.00,Friday,2019-10-25 00:20:00,60000.00,3.00,20.00,23781.00,23782


In [34]:
nfl.describe()

,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
count,82.00,82.00,82.00,82.00,82.00
mean,63221.41,239.47,19.51,13685.78,13686.78
std,234061.65,671.99,38.48,37547.38,37547.38
min,5000.00,0.10,1.00,10.00,11.00
25%,6000.00,5.00,1.00,111.00,112.00
50%,12000.00,25.00,3.00,689.00,690.00
75%,25000.00,100.00,20.00,4562.25,4563.25
max,1500000.00,5375.00,150.00,176470.00,176471.00


In [35]:
nfl.prizePool.sum()

5184155.55

In [36]:
doubleups = nfl['name'].str.contains('Double Up').sum()
print ('Double ups количество в неделю: {} \nПроцент от общего числа турниров: {:.0f}%'.format(doubleups, doubleups/nfl.name.count()*100))

Double ups количество в неделю: 25 
Процент от общего числа турниров: 30%


In [114]:
nhl.head(5)

,name,sport,gameCount,dayofweek,start,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
2,nhl $8k forecheck [20 entry max],NHL,7,Monday,2019-10-14 17:00:00,8000.00,4.00,20,2378,2379
3,nhl $5.2k power forward [2 entry max],NHL,7,Monday,2019-10-14 17:00:00,5200.00,88.00,2,67,68
4,nhl $15.6k sin bin [single entry],NHL,7,Monday,2019-10-14 17:00:00,15600.00,1500.00,1,11,12
5,nhl $40k blue line [$10k to 1st],NHL,7,Monday,2019-10-14 17:00:00,40000.00,222.00,6,200,201
6,nhl $6.85k crease master,NHL,7,Monday,2019-10-14 17:00:00,6850.00,691.00,1,11,12


In [207]:
nhl.describe()

,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
count,19.00,19.00,19.00,19.00,19.00
mean,19127.37,199.66,24.32,2197.47,2192.89
std,29067.91,411.19,45.69,3430.90,3432.72
min,5000.00,0.50,1.00,10.00,11.00
25%,5000.00,5.00,1.00,169.00,170.00
50%,8000.00,20.00,3.00,583.00,584.00
75%,14810.00,104.50,20.00,1703.50,1704.50
max,100000.00,1500.00,150.00,11890.00,11891.00


In [46]:
nhl.prizePool.sum()

140650.0

In [217]:
doubleups = nhl['name'].str.contains('Double Up').sum()
print ('Double ups количество в неделю: {} \nПроцент от общего числа турниров: {:.0f}%'.format(doubleups, doubleups/nhl.name.count()*100))

Double ups количество в неделю: 1 
Процент от общего числа турниров: 5%


In [64]:
print (json_data[17]['entryCount'], '\n', json_normalize(json_data[17]['prizes']))

11891 
     cash  maxFinish  minFinish tierPayoutDescriptions.Cash  value
0   1000          1          1                   $1,000.00   1000
1    350          2          2                     $350.00    350
2    200          3          3                     $200.00    200
3    100          4          4                     $100.00    100
4     50          5          5                      $50.00     50
5     30          7          6                      $30.00     30
6     20         10          8                      $20.00     20
7     15         13         11                      $15.00     15
8     10         18         14                      $10.00     10
9      8         28         19                       $8.00      8
10     6         48         29                       $6.00      6
11     5         73         49                       $5.00      5
12     4        103         74                       $4.00      4
13     3        183        104                       $3.00      3
14

In [62]:
json_normalize(json_data[19]['prizes'])

,cash,maxFinish,minFinish,tierPayoutDescriptions.Cash,value
0,25000,1,1,"$25,000.00",25000
1,10000,2,2,"$10,000.00",10000
2,5000,3,3,"$5,000.00",5000
3,3500,4,4,"$3,500.00",3500
4,2500,5,5,"$2,500.00",2500
5,2000,7,6,"$2,000.00",2000
6,1500,10,8,"$1,500.00",1500
7,1000,13,11,"$1,000.00",1000
8,700,20,14,$700.00,700
9,600,29,21,$600.00,600


In [105]:
rg_2018 = get_data_rg('01/01/2018', '12/31/2018')

In [106]:
with open('rg_2018.json', 'w', encoding='utf-8') as f:
    json.dump(rg_2018, f, ensure_ascii=False, indent=4)

In [107]:
!ls 

nflgame.ipynb  simple_eda.ipynb     sportsref_download.csv  Untitled.ipynb
rg_2018.json   simple_eda-v2.ipynb  testdata.json


In [108]:
rg_2018

[{'_id': '5b45e8a262fa9e4e0fe5184e',
  'rgPrizePool': 28071,
  'rgPrizeWinnerCount': 197,
  'entryCount': 2811,
  'prizePool': 80000,
  'slateType': 1,
  'gameCount': None,
  'siteSlateId': '22721',
  'start': '2018-01-02T00:00:00.000Z',
  'sport': 4,
  'maxEntriesPerUser': None,
  'entryFee': 33,
  'name': '$80K Tue NHL SUPER Goal ($20K to 1st)',
  'siteContestId': '215048594',
  '_slateId': '5b45e8a062fa9e4e0fe509ef',
  'complete': True,
  'prizes': [{'maxFinish': 2795, 'minFinish': 1694, 'cash': 0},
   {'maxFinish': 560, 'minFinish': 560, 'cash': 20},
   {'maxFinish': 559, 'minFinish': 446, 'cash': 60},
   {'maxFinish': 314, 'minFinish': 314, 'cash': 64},
   {'maxFinish': 313, 'minFinish': 225, 'cash': 70},
   {'maxFinish': 163, 'minFinish': 163, 'cash': 77.5},
   {'maxFinish': 160, 'minFinish': 99, 'cash': 80},
   {'maxFinish': 65, 'minFinish': 37, 'cash': 100},
   {'maxFinish': 35, 'minFinish': 35, 'cash': 125},
   {'maxFinish': 34, 'minFinish': 24, 'cash': 150},
   {'maxFinish': 

In [109]:
rg_20182 = get_data_rg('01/02/2018', '12/31/2018')

In [110]:
rg_20182

[{'_id': '5ae4aa4c5689223dfaa52f9a',
  'rgPrizePool': 1820,
  'rgPrizeWinnerCount': 91,
  'entryCount': 574,
  'slateType': 3,
  'gameCount': 5,
  'siteSlateId': '17071',
  'start': '2018-01-03T00:00:00.000Z',
  'sport': 3,
  'prizePool': 5000,
  'maxEntriesPerUser': 1,
  'maxEntries': 574,
  'entryFee': 10,
  'name': 'NBA Single Entry $10 Double Up',
  'siteContestId': '52118200',
  '_slateId': '5aff63489109304c57c2ad30',
  'winner': {'users': [], 'score': None},
  'complete': True,
  'prizes': [{'value': 20,
    'tierPayoutDescriptions': {'Cash': '$20.00'},
    'minFinish': 1,
    'maxFinish': 250,
    'cash': 20}]},
 {'_id': '5ae4aac2c074cf3dffdc9ca0',
  'rgPrizePool': 2300,
  'rgPrizeWinnerCount': 23,
  'entryCount': 345,
  'slateType': 3,
  'gameCount': 5,
  'siteSlateId': '17071',
  'start': '2018-01-03T00:00:00.000Z',
  'sport': 3,
  'prizePool': 8000,
  'maxEntriesPerUser': 1,
  'maxEntries': 345,
  'entryFee': 27,
  'name': 'NBA $8K Fast Break [Single Entry]',
  'siteContestId

In [111]:
rg = rg_2018+rg_20182

In [112]:
rg

[{'_id': '5b45e8a262fa9e4e0fe5184e',
  'rgPrizePool': 28071,
  'rgPrizeWinnerCount': 197,
  'entryCount': 2811,
  'prizePool': 80000,
  'slateType': 1,
  'gameCount': None,
  'siteSlateId': '22721',
  'start': '2018-01-02T00:00:00.000Z',
  'sport': 4,
  'maxEntriesPerUser': None,
  'entryFee': 33,
  'name': '$80K Tue NHL SUPER Goal ($20K to 1st)',
  'siteContestId': '215048594',
  '_slateId': '5b45e8a062fa9e4e0fe509ef',
  'complete': True,
  'prizes': [{'maxFinish': 2795, 'minFinish': 1694, 'cash': 0},
   {'maxFinish': 560, 'minFinish': 560, 'cash': 20},
   {'maxFinish': 559, 'minFinish': 446, 'cash': 60},
   {'maxFinish': 314, 'minFinish': 314, 'cash': 64},
   {'maxFinish': 313, 'minFinish': 225, 'cash': 70},
   {'maxFinish': 163, 'minFinish': 163, 'cash': 77.5},
   {'maxFinish': 160, 'minFinish': 99, 'cash': 80},
   {'maxFinish': 65, 'minFinish': 37, 'cash': 100},
   {'maxFinish': 35, 'minFinish': 35, 'cash': 125},
   {'maxFinish': 34, 'minFinish': 24, 'cash': 150},
   {'maxFinish': 